In [11]:
// Load the EnvConfigHelper class from our project
#load "../EnvConfigHelper.cs"

  // Load configuration from .env file
  var modelDeploymentName = EnvConfigHelper.AzureOpenAIDeploymentName;
  var azureOpenAIEndpoint = EnvConfigHelper.AzureOpenAIEndpoint;
  var azureOpenAIKey = EnvConfigHelper.AzureOpenAIKey;


// Display loaded configuration (without sensitive data)
Console.WriteLine("🔑 Configuration loaded successfully!");
Console.WriteLine($"Azure OpenAI Endpoint: {azureOpenAIEndpoint}");
Console.WriteLine($"Deployment Name: {modelDeploymentName}");
Console.WriteLine($"API Key: {(string.IsNullOrEmpty(azureOpenAIKey) ? "❌ Not loaded" : "✅ Loaded (hidden)")}");

✅ Loaded .env from: d:\Dev\VSC\semantic-kernel-in-action-1-fundamentals-3836112\src\.env
🔑 Configuration loaded successfully!
Azure OpenAI Endpoint: https://dis-openai-0705.openai.azure.com/
Deployment Name: gpt-4.1
API Key: ✅ Loaded (hidden)


In [6]:
#r "nuget: Microsoft.SemanticKernel, *"
#r "nuget: Microsoft.SemanticKernel.PromptTemplates.Handlebars, *"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.PromptTemplates.Handlebars;
using System.ComponentModel;
using System.IO;
using System.Text.Json;
using System.Net.Http;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;



Installed Packages Microsoft.SemanticKernel, 1.61.0 Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.61.0

In [7]:
public class WhatDateIsIt
{
  [KernelFunction, Description("Get the current date")]
  public string Date(IFormatProvider? formatProvider = null) =>
      DateTimeOffset.UtcNow.ToString("D", formatProvider);
}

In [12]:
async Task Execute()
  {
    
    var builder = Kernel.CreateBuilder();
    builder.Services.AddAzureOpenAIChatCompletion(
        modelDeploymentName,
        azureOpenAIEndpoint,
        azureOpenAIKey,
        modelId: "gpt-4.1"
    );
    var kernel = builder.Build();

    KernelFunction kernelFunctionRespondAsScientific =
        KernelFunctionFactory.CreateFromPrompt(
            "Respond to the user question as if you were a Scientific. Respond to it as you were him, showing your personality",
            functionName: "RespondAsScientific",
            description: "Responds to a question as a Scientific.");

    KernelFunction kernelFunctionRespondAsPoliceman =
        KernelFunctionFactory.CreateFromPrompt(
            "Respond to the user question as if you were a Policeman. Respond to it as you were him, showing your personality, humor and level of intelligence.",
            functionName: "RespondAsPoliceman",
            description: "Responds to a question as a Policeman.");

    KernelPlugin roleOpinionsPlugin =
        KernelPluginFactory.CreateFromFunctions(
            "RoleTalk",
            "Responds to questions or statements asuming different roles.",
            new[] {
                    kernelFunctionRespondAsScientific,
                    kernelFunctionRespondAsPoliceman
                  });
    kernel.Plugins.Add(roleOpinionsPlugin);
    kernel.Plugins.AddFromType<WhatDateIsIt>();

    string userPrompt = "I just woke up and found myself in the middle of nowhere, " +
        "do you know what date is it? and what would a policeman and a scientist do in my place?" +
        "Please provide me the date using the WhatDateIsIt plugin and the Date function, and then " +
        "the responses from the policeman and the scientist, on this order. " +
        "For this two responses, use the RoleTalk plugin and the RespondAsPoliceman and RespondAsScientific functions.";

    OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new()
    {
      ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
    };

    var result = await kernel.InvokePromptAsync(
        userPrompt,
        new(openAIPromptExecutionSettings));

    Console.WriteLine($"Result: {result}");
    Console.ReadLine();
  }

In [13]:
await Execute();

Result: Today is Saturday, 02 August 2025.

Here's how a policeman and a scientist would respond in your situation:

Policeman: "Absolutely, citizen! What’s your question? Don’t worry, you’re not in trouble… unless you ask me about my favorite donut flavor. (It’s jelly, by the way. Strictly off the record.) Fire away—whether it’s about law, safety, or if you’re lost on your way to a community bake sale, I’m all ears and ready to serve."

Scientist: "Absolutely! Please provide the user question you'd like me to answer as if I were Scientific, and I'll channel his personality and style for the response."

If you need more specific guidance from either the policeman or scientist, feel free to ask!
